In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [24]:
xlsx = pd.ExcelFile("DataSet.xlsx")
df = pd.read_excel(xlsx)



In [25]:
#Monthly return sp500
df['sp500ret']= df['SP500Index'].pct_change()

In [26]:
#Monthly return of bond index
df['bondret'] = df['BarclaysUSAggTreasury Index'].pct_change()


In [27]:
#Benchmark Portfolio, shows first 5 monthly returns
df['benchmarkret'] = .6*df['sp500ret'] + .4*df['bondret']
df['benchmarkret'].head()


0         NaN
1    0.008450
2    0.015780
3   -0.018664
4    0.069750
Name: benchmarkret, dtype: float64

In [28]:
#adding 1 to returns for caluclation purposes
df['1+sp500ret']= 1+ df['sp500ret']
df['1+bondret']=1+ df['bondret']
df['1+benchmarkret'] = 1 + df['benchmarkret']


In [29]:
#Total Compound Cumulative Returns 
#sp500CumRet = (df["SP500Index"][len(df.index)-1] / df['SP500Index'][0]) -1
#bondCumRet = (df['BarclaysUSAggTreasury Index'][len(df.index)-1] / df['BarclaysUSAggTreasury Index'][0])
df['sp500CumRet']= df['1+sp500ret'].cumprod()-1
df['bondCumRet'] = df['1+bondret'].cumprod()-1
df['benchmarkCumRet'] = df['1+benchmarkret'].cumprod()-1



In [30]:
#Annualized Returns, 
sp500AnnRet = (df["sp500CumRet"][len(df.index)-1])**(1/27.)-1
bondAnnRet = (df["bondCumRet"][len(df.index)-1])**(1/27.)-1
benchmarkAnnRet = (df["benchmarkCumRet"][len(df.index)-1])**(1/27.)-1
print sp500AnnRet, bondAnnRet, benchmarkAnnRet

0.0931989940909 0.0492857949103 0.0797343968425


In [31]:
#Annual Standard Deviation
sp500SD = np.std(df['sp500ret'])*np.sqrt(12)
bondSD = np.std(df['bondret'])*np.sqrt(12)
benchmarkSD = np.std(df['benchmarkret'])*np.sqrt(12)
print sp500SD, bondSD, benchmarkSD

0.14367923596 0.044111172661 0.086196221143


In [32]:
#annual return of risk free for sharpe ratio calculation
cumRetRF = (df['3m Tbill Index'][len(df.index)-1] /df['3m Tbill Index'][0]) - 1
rfAnnRet = cumRetRF**(1/27.) - 1
rfAnnRet

0.007419355012572959

In [33]:
#sharpe ratio
sp500SharpeRatio = (sp500AnnRet - rfAnnRet)/ sp500SD
bondSharpeRatio = (bondAnnRet - rfAnnRet)/ bondSD
benchmarkSharpeRatio = (benchmarkAnnRet - rfAnnRet )/ benchmarkSD
print sp500SharpeRatio, bondSharpeRatio, benchmarkSharpeRatio

0.597021820899 0.949111922721 0.838958377421


In [34]:
#summary chart for question 2
sp500CumRet = df["sp500CumRet"][len(df.index)-1]
bondCumRet = df["bondCumRet"][len(df.index)-1]
benchmarkCumRet = df["benchmarkCumRet"][len(df.index)-1]
dictSummary = {'Cum Return' : pd.Series([sp500CumRet, bondCumRet, benchmarkCumRet]),
               'Ann Return' : pd.Series([sp500AnnRet, bondAnnRet, benchmarkAnnRet]),
               'Ann Standard Dev' : pd.Series([sp500SD, bondSD, benchmarkSD]),
               'Sharpe Ratio' : pd.Series([sp500SharpeRatio, bondSharpeRatio, benchmarkSharpeRatio])}
                    

summary = pd.DataFrame(dictSummary)
summary1 = summary.set_index([['sp500','barclay','benchmark']])
summary1

,Ann Return,Ann Standard Dev,Cum Return,Sharpe Ratio
sp500,0.093199,0.143679,11.088659,0.597022
barclay,0.049286,0.044111,3.665493,0.949112
benchmark,0.079734,0.086196,7.935189,0.838958


In [35]:
#making new dataframe for questions 3+
df2 = df.iloc[:,0:10]

In [36]:
#Finding returns of the factor
for i in range(0,4):
    df2['factor{}ret'.format(i+1)] = df2['Factor{}'.format(i+1)].pct_change()


In [37]:
#Lagging returns of the factors for prediction purposes, shift factor returns one month down.
for i in range(0,4):
    df2['laggedf{}ret'.format(i+1)] = df2['factor{}ret'.format(i+1)].shift(1)

In [38]:
#10 year data regression

regress1 = smf.ols(formula='sp500ret ~ laggedf1ret', data=df2[2:123]).fit()
regress2 = smf.ols(formula='sp500ret ~ laggedf2ret', data=df2[2:123]).fit()
regress3 = smf.ols(formula='sp500ret ~ laggedf3ret', data=df2[2:123]).fit()
regress4 = smf.ols(formula='sp500ret ~ laggedf4ret', data=df2[2:123]).fit()
print regress3.summary()

                            OLS Regression Results                            
Dep. Variable:               sp500ret   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     12.19
Date:                Fri, 03 Nov 2017   Prob (F-statistic):           0.000675
Time:                        10:53:38   Log-Likelihood:                 226.61
No. Observations:                 121   AIC:                            -449.2
Df Residuals:                     119   BIC:                            -443.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0119      0.004      3.366      

In [39]:
#Calculating regression based off of last 3 years.(factor beta)
for i in range(0,4):
    df2['beta{}'.format(i+1)] = df2['sp500ret'].rolling(window=36).cov(other=df2['laggedf{}ret'.format(i+1)])/ df2['laggedf{}ret'.format(i+1)].rolling(window=36,center=False).var()
  


In [43]:
#Shifting betas down 1 cell to get rid of last month's look ahead bias
for i in range(0,4):
    df2['shiftbeta{}'.format(i+1)] = df2['beta{}'.format(i+1)].shift(1)

In [45]:
#average of sp500 returns from past 3 years, shifted down 1
df2['sp500RollingMean'] = df2['sp500ret'].shift(1).rolling(window=36).mean()

In [46]:
#average of lagged return from past 3 years, shifted down 1 
for i in range(0,4):
    df2['laggedf{}RollingMean'.format(i+1)] = df2['laggedf{}ret'.format(i+1)].shift(1).rolling(window=36).mean()


In [49]:
# finding intercept of linear regression
for i in range(0,4):
    df2['f{}intercept'.format(i+1)] = df2['sp500RollingMean'] - df2['shiftbeta{}'.format(i+1)]*df2['laggedf{}RollingMean'.format(i+1)]

In [50]:
writer = pd.ExcelWriter('Wellington2.xlsx')
df.to_excel(writer,'Sheet1')
df2.to_excel(writer,'Sheet2')
writer.save()

#Changing values to none because used 10 years worth of regression data to state factor 3 was good.
#df2.iloc[:123,18:31] = None 

In [51]:
#Predicted Return( y=mx+b) 
for i in range(0,4):
    df2['f{}PredictRet'.format(i+1)] = df2['f{}intercept'.format(i+1)] + df2['shiftbeta{}'.format(i+1)]*df2['laggedf{}ret'.format(i+1)]

In [52]:
#error squared, difference between predicted return and actual return
for i in range(0,4):
    df2['f{}errorSquare'.format(i+1)] = (df2['f{}PredictRet'.format(i+1)]- df2['sp500ret'])**2

,Date,SP500Index,BarclaysUSAggTreasury Index,3m Tbill Index,Factor1,Factor2,Factor3,Factor4,sp500ret,bondret,benchmarkret,1+sp500ret,1+bondret,1+benchmarkret,sp500CumRet,bondCumRet,benchmarkCumRet
120,2000-01-31,1919.84,960.87,513.749,24.95,169.3,363.66,56.7,-0.050242,0.002609,-0.029102,0.949758,1.002609,0.970898,4.424196,1.085900,2.775351
121,2000-02-29,1883.50,975.32,516.001,23.37,170.0,370.81,55.8,-0.018929,0.015038,-0.005342,0.981071,1.015038,0.994658,4.321523,1.117269,2.755184
122,2000-03-31,2067.76,994.69,518.689,24.11,171.0,359.35,54.9,0.097829,0.019860,0.066641,1.097829,1.019860,1.066641,4.842120,1.159318,3.005434
123,2000-04-28,2005.55,991.54,520.951,26.20,170.9,370.76,54.7,-0.030086,-0.003167,-0.019318,0.969914,0.996833,0.980682,4.666356,1.152480,2.928056


In [82]:
#Mean Squared Error, lowest MSE is factor chosen. (MSE from first date we have beta to the start date of strategy)
# using n-k-1 for denominator of MSE
#Strategy starts on 5/31/2000.(index=124)

print df2['f1errorSquare'][:124].mean(), df2['f2errorSquare'][:124].mean(), df2['f3errorSquare'][:124].mean(), df2['f4errorSquare'][:124].mean()

#Factor 3 is best.

0.00172503773421 0.00156975621483 0.00135656532466 0.00154052369897


In [83]:
writer = pd.ExcelWriter('Wellington2.xlsx')
df.to_excel(writer,'Sheet1')
df2.to_excel(writer,'Sheet2')
writer.save()

In [84]:
#Rebalance formula

df2['equityWeight']=.6


for i in range(124,len(df.index)):
    rebalanceFormula = 1.5 - (1/(1+np.exp(df2['f3PredictRet'][i])))

    if df2['f3PredictRet'][i] >= 0:
        if df2['equityWeight'][i-1]* rebalanceFormula >= .7:
            df2['equityWeight'][i] = .7
        else:
            df2['equityWeight'][i] = df2['equityWeight'][i-1]* rebalanceFormula
    else:
        if df2['equityWeight'][i-1]* rebalanceFormula <= .5:
            df2['equityWeight'][i] = .5
        else:
            df2['equityWeight'][i] = df2['equityWeight'][i-1]*rebalanceFormula


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [85]:
#calculating bondweight of portfolio
df2['bondWeight'] = 1 - df2['equityWeight']


In [86]:
#Setting first couple of rows of equity/bond weights as None,  trading strategy didn't start yet(Strategy starts 5/31/2000)
df2['equityWeight'][:124] = None
df2['bondWeight'][:124] = None


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
# portfolio return
df2['portfolioRet'] = df2['bondWeight']*df2['bondret'] + df2['equityWeight']*df2['sp500ret']

In [88]:
#alpha
df2['alpha'] = df2['portfolioRet'] - df['benchmarkret']


In [103]:
#cumulative alpha
cumAlpha = (1 + df2['alpha']).prod() - 1
cumAlpha

0.055231359474959385

In [90]:
#annualized alpha
n = 201./12 # #of periods
annualAlpha = (1+ cumAlpha)**(1/n) - 1
annualAlpha


0.0032147108200126784

In [91]:
#tracking error
#df2['alpha'].std()#, also tracking error 
numMonths = 201
trackingError = ((df2['alpha']**2).sum()/numMonths)**(1/2.)
trackingError
            

0.0023920753984750247

In [92]:
#Information Ratio
infoRatio = annualAlpha/ trackingError
infoRatio

1.343900289289415

In [93]:
#random initial portfolio value
df2['portfolioValue'] = 1000000

In [94]:
for i in range(124, len(df2.index)):
    df2['portfolioValue'][i] = df2['portfolioValue'][i-1]* (1+df2['portfolioRet'][i])

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [95]:
# maxdrawdown = (trough - peak)/peak, max loss could possibly earn if invested in this portfolio
#cumulative portfolio return, maximum of cumulative portfolio return
df2['maxPortfolioValue'] = df2['portfolioValue'].cummax() #max value of portfolio up to that date , peak
df2['maxDrawdown'] = (df2['portfolioValue']) / (df2['maxPortfolioValue']) - 1 #maximum value potentially could have lost up to that date
df2['maxDrawdown'].min() 

-0.32880045067146413

In [96]:
#Bonus questions, trading costs
tradingCost = .005*2 #have to trade bond index and stock index
df2['portRetTradingCost'] = None
df2['equityWeight'][122] = .6 # to make loop work
for i in range(123, len(df.index)):
    if df2['equityWeight'][i-1] != df2['equityWeight'][i]:
        df2['portRetTradingCost'][i] = df2['portfolioRet'][i] - tradingCost
    else:
        df2['portRetTradingCost'][i] = df2['portfolioRet'][i]


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [97]:
#check if strategy still returns alpha with previous code
df2['alphaTC'] = df2['portRetTradingCost'] - df['benchmarkret']


In [98]:
#cumulative alpha with trading cost
cumAlphaTC = (1 + df2['alphaTC']).prod() - 1
cumAlphaTC

-0.81447916197686032

In [99]:
#annualized alpha with trading cost
n = 201./12 #of periods 
annualAlphaTC = (1+ cumAlphaTC)**(1/n) - 1
annualAlphaTC

-0.095680382615073278

In [100]:
#question 4

summary2 = pd.DataFrame({'Portfolio Performance' : [cumAlpha,annualAlpha,trackingError,infoRatio,df2['maxDrawdown'].min()]})
summary2.set_index([['Cum Alpha', 'Ann Alpha', 'Tracking Error', 'Info Ratio', 'Max Drawdown']])

,Portfolio Performance
Cum Alpha,0.055231
Ann Alpha,0.003215
Tracking Error,0.002392
Info Ratio,1.343900
Max Drawdown,-0.328800


In [101]:
writer = pd.ExcelWriter('Wellington2.xlsx')
df.to_excel(writer,'Sheet1')
df2.to_excel(writer,'Sheet2')
writer.save()